In [1]:
import os
PATH = "/home/mdobrzanski/playground/IET/NLP/ustawy/"
%config Completer.use_jedi = False

In [2]:
import re
import time
from collections import defaultdict, Counter, OrderedDict
from functools import reduce
import math
import pandas as pd
import requests
import collections
import tensorflow as tf
import torch as pytorch
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, confusion_matrix, precision_score, recall_score
from imblearn.over_sampling import RandomOverSampler
import fasttext
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from lime.lime_text import LimeTextExplainer

In [3]:
import ktrain

In [4]:
pytorch.cuda.is_available()

True

In [5]:
# pytorch.cuda.get_device_name(0)

In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [7]:
# pip install torch==1.8.0

In [8]:
from ipywidgets import IntProgress

# 1

In [9]:
dataset_1 = load_dataset("poleval2019_cyberbullying", 'task01')

Using the latest cached version of the module from /home/mdobrzanski/.cache/huggingface/modules/datasets_modules/datasets/poleval2019_cyberbullying/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450 (last modified on Tue Dec  7 18:31:15 2021) since it couldn't be found locally at poleval2019_cyberbullying., or remotely on the Hugging Face Hub.
Reusing dataset poleval2019_cyber_bullying (/home/mdobrzanski/.cache/huggingface/datasets/poleval2019_cyber_bullying/task01/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450)


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
dataset_2 = load_dataset("poleval2019_cyberbullying", 'task02')

Using the latest cached version of the module from /home/mdobrzanski/.cache/huggingface/modules/datasets_modules/datasets/poleval2019_cyberbullying/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450 (last modified on Tue Dec  7 18:31:15 2021) since it couldn't be found locally at poleval2019_cyberbullying., or remotely on the Hugging Face Hub.
Reusing dataset poleval2019_cyber_bullying (/home/mdobrzanski/.cache/huggingface/datasets/poleval2019_cyber_bullying/task02/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450)


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
dataset_1

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10041
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

In [12]:
dataset_2

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10041
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

### Dataset 1

In [13]:
print('Train O: ', dataset_1['train']['label'].count(0))
print('Train 1: ', dataset_1['train']['label'].count(1))
print('Ratio train: ', dataset_1['train']['label'].count(1)/len(dataset_1['train']['label']))

Train O:  9190
Train 1:  851
Ratio train:  0.08475251468977194


In [14]:
print('Test O: ', dataset_1['test']['label'].count(0))
print('Test 1: ', dataset_1['test']['label'].count(1))
print('Ratio test: ', dataset_1['test']['label'].count(1)/len(dataset_1['test']['label']))

Test O:  866
Test 1:  134
Ratio test:  0.134


### Dataset 2

In [15]:
print('Train O: ', dataset_2['train']['label'].count(0))
print('Train 1: ', dataset_2['train']['label'].count(1))
print('Train 2: ', dataset_2['train']['label'].count(2))

Train O:  9190
Train 1:  253
Train 2:  598


In [16]:
print('Test O: ', dataset_2['test']['label'].count(0))
print('Test 1: ', dataset_2['test']['label'].count(1))
print('Test 2: ', dataset_2['test']['label'].count(2))

Test O:  866
Test 1:  25
Test 2:  109


### Prepare dataset 1

In [17]:
X1_tr_base = dataset_1['train']['text']
y1_tr_base = dataset_1['train']['label']

X1_tst_base = dataset_1['test']['text']
y1_tst_base = dataset_1['test']['label']

In [18]:
vect1 = TfidfVectorizer()

vect1.fit(X1_tr_base)

TfidfVectorizer()

In [19]:
X1_tr = vect1.transform(X1_tr_base)
X1_tst = vect1.transform(X1_tst_base)

In [20]:
ros = RandomOverSampler(random_state=0)
X1_tr, y1_tr = ros.fit_resample(X1_tr, y1_tr_base)

### Prepare dataset 2

In [21]:
X2_tr_base = dataset_2['train']['text']
y2_tr_base = dataset_2['train']['label']

X2_tst_base = dataset_2['test']['text']
y2_tst_base = dataset_2['test']['label']

In [22]:
vect2 = TfidfVectorizer()

vect2.fit(X2_tr_base)

TfidfVectorizer()

In [23]:
X2_tr = vect2.transform(X2_tr_base)
X2_tst = vect2.transform(X2_tst_base)

In [24]:
ros = RandomOverSampler(random_state=0)
X2_tr, y2_tr = ros.fit_resample(X2_tr, y2_tr_base)

# 2

### Bayesian classifier with TF * IDF weighting.

Dataset 1

In [25]:
bayes1 = MultinomialNB()
bayes1.fit(X1_tr, y1_tr)

MultinomialNB()

In [26]:
bayes1.score(X1_tst, y1_tst_base)

0.826

Dataset 2

In [27]:
bayes2 = MultinomialNB()
bayes2.fit(X2_tr, y2_tr)

MultinomialNB()

In [28]:
bayes2.score(X2_tst, y2_tst_base)

0.784

### Fasttext text classifier

In [29]:
with open('data_1_tr.txt', "w") as f:
    for text, y in zip(X1_tr, y1_tr):
        f.write("__label__{} {}\n".format(y, text))

with open('data_1_tst.txt', "w") as f:
    for text, y in zip(X1_tst, y1_tst_base):
        f.write("__label__{} {}\n".format(y, text))
        
with open('data_2_tr.txt', "w") as f:
    for text, y in zip(X2_tr, y2_tr):
        f.write("__label__{} {}\n".format(y, text))
        
with open('data_2_tst.txt', "w") as f:
    for text, y in zip(X2_tst, y2_tst_base):
        f.write("__label__{} {}\n".format(y, text))

In [30]:
fasttext1 = fasttext.train_supervised('data_1_tr.txt')

In [31]:
fasttext1.test('data_1_tst.txt')

(1000, 0.873, 0.873)

In [32]:
fasttext2 = fasttext.train_supervised('data_2_tr.txt')

In [33]:
fasttext2.test('data_2_tst.txt')

(1000, 0.87, 0.87)

### Transformer classifier 

In [34]:
# model = ClassificationModel('bert', 'bert-base-uncased')

# tr_df1 = pd.DataFrame(list(map(list, zip(*[X1_tr_base, y1_tr]))), columns=['text', 'label'])

# model.train_model(tr_df1)

# model.predict(tr_df1['text'][0])

In [35]:
ros = RandomOverSampler(random_state=0, sampling_strategy=0.3)
X1_tr_base_res, y1_tr_res = ros.fit_resample(list(map(lambda x: [x],X1_tr_base)), y1_tr_base)

In [36]:
X1_tr_base_res = list(map(lambda x: x[0], X1_tr_base_res))

In [37]:
transformer1 = ktrain.text.Transformer(
        'bert-base-uncased', 
        maxlen=128,
        class_names=list(set(y1_tr_res))
    )

In [40]:
train1 = transformer1.preprocess_train(X1_tr_base_res, y1_tr_res)

preprocessing train...
language: pl
train sequence lengths:
	mean : 12
	95percentile : 21
	99percentile : 24


Is Multi-Label? False


In [41]:
classifier1 = transformer1.get_classifier()

learner1 = ktrain.get_learner(
        classifier1,
        train_data=train1,
        batch_size=8)

In [42]:
# learner1.fit_onecycle(0.0001, 5)

In [43]:
# learner1.save_model('model1.pkl')

In [44]:
learner1.load_model('model1.pkl')

In [45]:
# learner1.fit_onecycle(0.0001, 5)

In [46]:
ros = RandomOverSampler(random_state=0, sampling_strategy='minority')
X2_tr_base_res, y2_tr_res = ros.fit_resample(list(map(lambda x: [x],X2_tr_base)), y2_tr_base)

In [47]:
X2_tr_base_res = list(map(lambda x: x[0], X2_tr_base_res))

In [48]:
transformer2 = ktrain.text.Transformer(
        'bert-base-uncased', 
        maxlen=200,
        class_names=list(set(y2_tr_res))
    )

In [49]:
train2 = transformer2.preprocess_train(X2_tr_base_res, y2_tr_res)

preprocessing train...
language: pl
train sequence lengths:
	mean : 12
	95percentile : 20
	99percentile : 23


Is Multi-Label? False


In [50]:
classifier2 = transformer2.get_classifier()

learner2 = ktrain.get_learner(
        classifier2,
        train_data=train2,
        batch_size=4
    )

In [51]:
# learner2.fit_onecycle(0.0001, 1)

In [52]:
# learner2.save_model('model2.pkl')

In [53]:
learner2.load_model('model2.pkl')

# 3

#### Task 1

In [54]:
results_bayes1 = bayes1.predict(X1_tst)

In [55]:
results_fasttext1 = fasttext1.predict(X1_tst_base)[0]

results_fasttext1 = list(map(lambda x: int(x[0].replace('__label__', '')), results_fasttext1))

In [56]:
test1 = transformer1.preprocess_train(X1_tst_base, y1_tst_base)

preprocessing train...
language: pl
train sequence lengths:
	mean : 12
	95percentile : 21
	99percentile : 23


Is Multi-Label? False


In [57]:
results_transformer1 = learner1.predict(test1)

In [58]:
results_transformer1 = (results_transformer1[:,1]>0.5).astype(int)

In [59]:
accuracy_score(results_bayes1, y1_tst_base)

0.826

In [60]:
accuracy_score(results_fasttext1, y1_tst_base)

0.866

In [61]:
accuracy_score(results_transformer1, y1_tst_base)

0.862

In [62]:
f1_score(results_bayes1, y1_tst_base)

0.46296296296296297

In [63]:
f1_score(results_fasttext1, y1_tst_base)

0.0

In [64]:
f1_score(results_transformer1, y1_tst_base)

0.45238095238095233

In [65]:
confusion_matrix(results_bayes1, y1_tst_base)

array([[751,  59],
       [115,  75]])

In [66]:
confusion_matrix(results_fasttext1, y1_tst_base)

array([[866, 134],
       [  0,   0]])

In [67]:
confusion_matrix(results_transformer1, y1_tst_base)

array([[805,  77],
       [ 61,  57]])

#### Task 2

In [68]:
results_bayes2 = bayes2.predict(X2_tst)

In [69]:
results_fasttext2 = fasttext2.predict(X2_tst_base)[0]

results_fasttext2 = list(map(lambda x: int(x[0].replace('__label__', '')), results_fasttext2))

In [70]:
test2 = transformer2.preprocess_train(X2_tst_base, y2_tst_base)

preprocessing train...
language: pl
train sequence lengths:
	mean : 12
	95percentile : 21
	99percentile : 23


Is Multi-Label? False


In [71]:
results_transformer2 = learner2.predict(test2)

In [72]:
results_transformer2 = (results_transformer2[:,1]>0.5).astype(int)

In [73]:
accuracy_score(results_bayes2, y2_tst_base)

0.784

In [74]:
accuracy_score(results_fasttext2, y2_tst_base)

0.866

In [75]:
accuracy_score(results_transformer2, y2_tst_base)

0.844

In [76]:
f1_score(results_bayes2, y2_tst_base, average='micro')

0.7840000000000001

In [77]:
f1_score(results_fasttext2, y2_tst_base, average='micro')

0.866

In [78]:
f1_score(results_transformer2, y2_tst_base, average='micro')

0.844

In [79]:
f1_score(results_bayes2, y2_tst_base, average='macro')

0.48422604436085454

In [80]:
f1_score(results_fasttext2, y2_tst_base, average='macro')

0.3093962129331904

In [81]:
f1_score(results_transformer2, y2_tst_base, average='macro')

0.3639339975419053

In [82]:
confusion_matrix(results_bayes2, y2_tst_base)

array([[727,   9,  46],
       [ 63,  10,  16],
       [ 76,   6,  47]])

In [83]:
confusion_matrix(results_fasttext2, y2_tst_base)

array([[866,  25, 109],
       [  0,   0,   0],
       [  0,   0,   0]])

In [84]:
confusion_matrix(results_transformer2, y2_tst_base)

array([[838,  19,  98],
       [ 28,   6,  11],
       [  0,   0,   0]])

# 4